# Webscrape IRCC

In [13]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

### Set the URL you want to webscrape from

In [14]:
CND_SITE = 'https://www.canada.ca'
IRCC_SUFFIX='/en/immigration-refugees-citizenship/'
filter = re.compile('^' + IRCC_SUFFIX + '.*')
exclude_some = re.compile("#")
url = CND_SITE + IRCC_SUFFIX

In [19]:
def get_hrefs(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  return set(sorted([a['href'] for a in soup.findAll('a', href=filter) if not exclude_some.search(a['href'])]))

### Connect to the site - get the first page and loop through links to get children
> #### Paramateres
>> #### SCRAPE_DEPTH: determines how many layers we loop through - less: quicker; more: more pages; setting it at 2 returns a bit over 5000 links; setting it too high make it take longer to complete
>> #### WEBCALLS_WITH_NO_BREAK: determines how often we take a one-second break from scraping. It helps to prevent from flooding the site and risk our IP be banned

In [21]:
scraped_uris = get_hrefs(url)
# in my experience two iterations returns over 5000 links
#   one can experience with more if necessary
SCRAPE_DEPTH = 1
WEBCALLS_WITH_NO_BREAK = 10

for i in range(SCRAPE_DEPTH):
  new_uris = set()
  web_calls=0
  for suffix in scraped_uris:
    new_uris |= get_hrefs(CND_SITE+suffix)
    # take a second between every few site calls to not flood the site
    web_calls += 1
    if web_calls >= WEBCALLS_WITH_NO_BREAK:
      time.sleep(1)
      web_calls = 0
  scraped_uris |= new_uris

len(scraped_uris)

629

## To download the whole data set, let's do a for loop through all a tags

In [24]:
count = 10000
for ahref in scraped_uris:
  download_url = CND_SITE + ahref
  count += 1
  urllib.request.urlretrieve(download_url,'./file-'+str(count)) 
  time.sleep(1)
